In [19]:
import numpy as np
import requests as r
from urllib.request import urlopen
from bs4 import BeautifulSoup

## 怎麼抓 ArcGIS Map Data  
要知道的是 ArcGIS 其實提供REST API 來幫助我們來抓資料。  
那要怎麼找到其 API 的 Ｑuery 功能呢？  
首先 我們要知道 我們的網站 包含了什麼內容  

> https://txdshs.maps.arcgis.com/apps/opsdashboard/index.html#/ed483ecd702b4298ab01e8b9cafc8b83  

你看到了喔！ 其實，我們可以看到我們如果要在ArcGIS 上發表 GIS Map，其實都需要提供一個 ArcGIS server APP ID. 從上述的網址，我們可以看到唯一有可能的APP ID，就是最後面那一串 ed483ecd702b4298ab01e8b9cafc8b83。所以，我們可以把ArcGIS REST的網址加上去，最後再加上一個 "/data"。因此，我們可以得到  

> https://www.arcgis.com/sharing/rest/content/items/ed483ecd702b4298ab01e8b9cafc8b83/data

當進入這個網頁，因為這個ArcGIS service 是dashboard. 所以我們要找的是有什麼id我們可以使用的。我查了一下，我找到...

> "id":  
> a4d3aa1f-2299-4a20-a89b-bcf5bba4de59   ! Don't think this one is the one i am looking    
> f6623ad0-d45a-474e-8313-0d0c6cb57f15   ! Don't think this one is the one i am looking  
> 4fb2b8f9-0e3a-4392-8d7e-366851993444   ! serviceDataset
> 4fb2b8f9-0e3a-4392-8d7e-366851993444#COVID19County_ViewLayer_414
> 
> df4910f0-06d8-4d9f-aff7-25ff1d07cff9   ! Cases Reported  
> 69846f80-7b25-4da4-a84a-dd32412984ab   ! Counties Reporting Cases  
> a5d281d4-0d36-4f4e-8de2-9be05b72015c
> 7a7b9dc4-6b0a-4cc1-a964-7a0a4f339a96
> 8bb96480-29b1-4db4-83c0-1ad89829ca68   ! Public Lab. 
> 68870376-22ea-4575-b442-f4310e2abbca   ! Private Labs
> 6b80815b-05f9-4d5d-96b8-26e98d39cd43
> 352fa542-6fa0-4aee-912a-1f2b98573ec9
> 
> "id":"4fb2b8f9-0e3a-4392-8d7e-366851993444","width":1,"height":1,"tabName":"Case Numbers By County"
> "id":"4c8fdae0-b3ac-4cdc-9135-af0355d66441","width":1,"height":1,"tabName":"Deaths By County"

那我們來把網址接上，最後得到兩個可能性
https://www.arcgis.com/sharing/rest/content/items/f8fd6fb4cfdd4daaae24cbc2b2360fa5/data  
https://www.arcgis.com/sharing/rest/content/items/52f83dfc78114effbb94f62484cdb776/data

從這兩個可能性，如果我們點進去面那個網址，我們可以看到...  
網址顯示 “operationalLayers" 然後 id, layerType, 還有url。  
理論上，這個URL 應該就是我們想要得到的ArcGIS REST url。  
>  https://services5.arcgis.com/ACaLB9ifngzawspq/arcgis/rest/services/COVID19County_ViewLayer/FeatureServer/0
>  https://services5.arcgis.com/ACaLB9ifngzawspq/arcgis/rest/services/COVID19County_Death/FeatureServer/0

點進最下面，我們就會出現 query 這個字眼。點擊 "query"，就會連到新的網址，這個就是我們最後的目的地。

## 怎麼使用query  
讓我們來看看 Document 的例子怎麼說  

https://services.myserver.com/OrgID/arcgis/rest/services/DateTimeIntervalQuery/FeatureServer/0/query?where=date_time > CURRENT_TIMESTAMP - INTERVAL '1' DAY&returnGeometry=false&returnCountOnly=true&resultType=&f=pjson  

https://services.myserver.com/OrgID/arcgis/rest/services/DateTimeIntervalQuery/FeatureServer/0/query?where=date_time > CURRENT_TIMESTAMP - INTERVAL '1' DAY&returnGeometry=false&returnCountOnly=true&resultType=&f=pjson  

https://services.myserver.com/OrgID/arcgis/rest/services/DateTimeIntervalQuery/FeatureServer/0/query?where=date_time > CURRENT_TIMESTAMP + INTERVAL '1 04' DAY TO HOUR&returnGeometry=false&returnCountOnly=true&resultType=&f=pjson  

因此，我們的例子可以得到的是
https://services5.arcgis.com/ACaLB9ifngzawspq/ArcGIS/rest/services/COVID19County_ViewLayer/FeatureServer/0/query?where=ObjectID%3E-1&outFields=*&ReturnGeometry=false&f=pgeojson

ps: ObjectID  可以用 0=0, 1=1, 和 ObjectID > -1 來取代 ObjectID%3E-1



## 開始爬我的資料  
fid = 'https://services5.arcgis.com/ACaLB9ifngzawspq/ArcGIS/rest/services/COVID19County_ViewLayer/FeatureServer/0/query?where=ObjectID%3E-1&outFields=*&ReturnGeometry=false&f=pgeojson'

req = r.get(url)

In [32]:
fid = 'https://services5.arcgis.com/ACaLB9ifngzawspq/ArcGIS/rest/services/COVID19County_ViewLayer/FeatureServer/0/query?where=ObjectID%3E-1&outFields=*&ReturnGeometry=false&f=pgeojson'
html= r.get(fid)
html.encoding='utf-8'
print(html.text)

{
  "type" : "FeatureCollection", 
  "features" : [
    {
      "type" : "Feature", 
      "id" : 1, 
      "geometry" : null, 
      "properties" : {
        "OBJECTID_1" : 1, 
        "OBJECTID" : 1, 
        "County" : "Anderson", 
        "FIPS" : "48001", 
        "COUNTYFP10" : "001", 
        "Shape_Leng" : 373739.497368, 
        "Count_" : null, 
        "LastUpdate" : 1585414447736, 
        "Shape__Area" : 2780384736.81836, 
        "Shape__Length" : 373739.497367559, 
        "Deaths" : null
      }
    }, 
    {
      "type" : "Feature", 
      "id" : 2, 
      "geometry" : null, 
      "properties" : {
        "OBJECTID_1" : 2, 
        "OBJECTID" : 2, 
        "County" : "Andrews", 
        "FIPS" : "48003", 
        "COUNTYFP10" : "003", 
        "Shape_Leng" : 256968.659368, 
        "Count_" : 1, 
        "LastUpdate" : 1585414447736, 
        "Shape__Area" : 3872439726.24023, 
        "Shape__Length" : 256968.659368344, 
        "Deaths" : null
      }
    }, 
    {


In [ ]:
req.text

In [25]:
# url = 'https://txdshs.maps.arcgis.com/apps/opsdashboard/index.html#/ed483ecd702b4298ab01e8b9cafc8b83'
url = 'https://txdshs.maps.arcgis.com/apps/opsdashboard/index.html'
req = r.get(url)

In [26]:
with open('abc.txt','w') as f:
    f.write(req.text)

In [ ]:
req.text

In [10]:
info.find_all('nav', {'class':'feature-list'})

[]

In [27]:
json = {"operationalLayers":[
    {"id":"Cazuri_judet_COVID_19_WFL1_4947",
     "layerType":"ArcGISFeatureLayer",
     "url":"https://services7.arcgis.com/I8e17MZtXFDX9vvT/arcgis/rest/services/Cazuri_judet_COVID_19_WFL1/FeatureServer/0",
     "visibility":true,
     "opacity":1,
     "title":"Judete",
     "itemId":"605c47cfed524ebabf7e6badd7fcabf2",
     "layerDefinition":{
         "drawingInfo":{
             "renderer":{
                 "type":"simple",
                 "symbol":{
                     "color":null,
                     "outline":{
                         "color":[153,153,153,255],
                         "width":0.7,
                         "type":"esriSLS",
                         "style":"esriSLSSolid"
                               },
                     "type":"esriSFS",
                     "style":"esriSFSSolid"}
                           },
                "labelingInfo":[{"labelExpression":null,"labelExpressionInfo":{"expression":"$feature[\"den_diacritice\"]","value":"{den_diacritice}"},"useCodedValues":true,"maxScale":0,"minScale":0,"where":null,"labelPlacement":"esriServerPolygonPlacementAlwaysHorizontal","symbol":{"color":[211,211,211,255],"type":"esriTS","backgroundColor":null,"borderLineColor":null,"haloSize":0,"haloColor":null,"horizontalAlignment":"center","rightToLeft":false,"angle":0,"xoffset":0,"yoffset":0,"text":"","rotated":false,"kerning":true,"font":{"size":11.25,"style":"italic","decoration":"none","weight":"normal","family":"Arial"}}}]},"definitionExpression":"den_diacritice <> 'Ilfov'"},"showLabels":true,"disablePopup":true,"showLegend":false},{"id":"Cazuri_judet_COVID_19_WFL1_4947_2291","layerType":"ArcGISFeatureLayer","url":"https://services7.arcgis.com/I8e17MZtXFDX9vvT/arcgis/rest/services/Cazuri_judet_COVID_19_WFL1/FeatureServer/0","visibility":true,"opacity":1,"title":"Ilfov","layerDefinition":{"drawingInfo":{"renderer":{"type":"simple","symbol":{"color":null,"outline":{"color":[153,153,153,255],"width":0.7,"type":"esriSLS","style":"esriSLSSolid"},"type":"esriSFS","style":"esriSFSSolid"}},"labelingInfo":[{"labelExpression":null,"labelExpressionInfo":{"expression":"$feature[\"den_diacritice\"]","value":"{den_diacritice}"},"useCodedValues":true,"maxScale":0,"minScale":1275858,"where":null,"labelPlacement":"esriServerPolygonPlacementAlwaysHorizontal","symbol":{"color":[211,211,211,255],"type":"esriTS","backgroundColor":null,"borderLineColor":null,"haloSize":0,"haloColor":null,"horizontalAlignment":"center","rightToLeft":false,"angle":0,"xoffset":0,"yoffset":0,"text":"","rotated":false,"kerning":true,"font":{"size":11.25,"style":"italic","decoration":"none","weight":"normal","family":"Arial"}}}]},"definitionExpression":"den_diacritice = 'Ilfov'"},"showLabels":true,"showLegend":false},{"id":"Coronavirus_romania_3184","layerType":"ArcGISFeatureLayer","url":"https://services7.arcgis.com/I8e17MZtXFDX9vvT/arcgis/rest/services/Coronavirus_romania/FeatureServer/0","visibility":false,"opacity":0.8,"title":"Cazuri confirmate la nivel de județ","itemId":"4a5cdc95ad584442a7fee1709076cb07","refreshInterval":5,"layerDefinition":{"drawingInfo":{"renderer":{"visualVariables":[{"type":"sizeInfo","field":"Cazuri_confirmate","valueExpression":null,"valueUnit":"unknown","minSize":0.75,"maxSize":33.75,"minDataValue":0,"maxDataValue":43}],"authoringInfo":{"visualVariables":[{"type":"sizeInfo","minSliderValue":0,"maxSliderValue":43}]},"type":"classBreaks","field":"Cazuri_confirmate","minValue":-9007199254740991,"classBreakInfos":[{"symbol":{"color":[255,0,0,170],"size":9,"angle":0,"xoffset":0,"yoffset":0,"type":"esriSMS","style":"esriSMSCircle","outline":{"color":null,"width":0.75,"type":"esriSLS","style":"esriSLSNull"}},"classMaxValue":9007199254740991}]}}},"popupInfo":{"title":"{Judete}","fieldInfos":[{"fieldName":"OBJECTID","visible":false,"isEditable":false,"label":"OBJECTID"},{"fieldName":"Judete","visible":true,"isEditable":true,"label":"Judete"},{"fieldName":"Cazuri_confirmate","visible":true,"isEditable":true,"label":"Cazuri confirmate","format":{"places":0,"digitSeparator":true}},{"fieldName":"SIRUTA_judet","visible":false,"isEditable":true,"label":"SIRUTA_judet","format":{"places":0,"digitSeparator":true}},{"fieldName":"Regiune_dezvoltare","visible":false,"isEditable":true,"label":"Regiune_dezvoltare"},{"fieldName":"Populatie","visible":false,"isEditable":true,"label":"Populatie","format":{"places":2,"digitSeparator":true}},{"fieldName":"Persoane_in_carantina","visible":true,"isEditable":true,"label":"Persoane în carantină","format":{"places":0,"digitSeparator":true}},{"fieldName":"Persoane_izolate","visible":true,"isEditable":true,"label":"Persoane izolate","format":{"places":0,"digitSeparator":true}},{"fieldName":"Persoane_decedate","visible":true,"isEditable":true,"label":"Persoane decedate","format":{"places":0,"digitSeparator":true}},{"fieldName":"Persoane_vindecate","visible":true,"isEditable":true,"label":"Persoane vindecate","format":{"places":0,"digitSeparator":true}},{"fieldName":"Probe_in_asteptare","visible":true,"isEditable":true,"label":"Probe în așteptare","format":{"places":0,"digitSeparator":true}},{"fieldName":"Cazuri_infirmate","visible":true,"isEditable":true,"label":"Cazuri infirmate","format":{"places":0,"digitSeparator":true}},{"fieldName":"GlobalID","label":"GlobalID","isEditable":false,"visible":false},{"fieldName":"EditDate","label":"EditDate","isEditable":false,"visible":false,"format":{"dateFormat":"shortDateShortTime","timezone":"utc"}}],"description":"Număr cazuri confirmate:<font color='#ff0000'> {Cazuri_confirmate} <\/font><b><br /><\/b>Persoane izolate: <font color='#4169e1'>{Persoane_izolate}<\/font><span style='font-weight: bold;'><br /><\/span>Persoane în carantină<span style='color: rgb(255, 0, 0);'><span style='font-weight: bold;'><span style='color: rgb(76, 76, 76);'><\/span><\/span><\/span>: <font color='#ffa500'>{Persoane_in_carantina}<\/font><span style='color: rgb(255, 0, 0);'><span style='font-weight: bold;'><br /><\/span><\/span>Persoane decedate: <font color='#ffffff'>{Persoane_decedate}<\/font>","showAttachments":false,"mediaInfos":[]}},{"id":"Coronavirus_romania_3184_4454","layerType":"ArcGISFeatureLayer","url":"https://services7.arcgis.com/I8e17MZtXFDX9vvT/arcgis/rest/services/Coronavirus_romania/FeatureServer/0","visibility":true,"opacity":0.8,"title":"Persoane în carantină la nivel de județ","refreshInterval":5,"layerDefinition":{"drawingInfo":{"renderer":{"visualVariables":[{"type":"sizeInfo","field":"Persoane_in_carantina","valueExpression":null,"valueUnit":"unknown","minSize":9,"maxSize":37.5,"minDataValue":20,"maxDataValue":150}],"authoringInfo":{"visualVariables":[{"type":"sizeInfo","minSliderValue":18,"maxSliderValue":477}]},"type":"classBreaks","field":"Persoane_in_carantina","minValue":-9007199254740991,"classBreakInfos":[{"symbol":{"color":[227,139,79,255],"size":9,"angle":0,"xoffset":0,"yoffset":0,"type":"esriSMS","style":"esriSMSCircle","outline":{"color":[255,255,255,64],"width":0.75,"type":"esriSLS","style":"esriSLSSolid"}},"classMaxValue":9007199254740991}]}}},"popupInfo":{"title":"{Judete}","fieldInfos":[{"fieldName":"OBJECTID","visible":false,"isEditable":false,"label":"OBJECTID"},{"fieldName":"Judete","visible":true,"isEditable":true,"label":"Judete"},{"fieldName":"Cazuri_confirmate","visible":false,"isEditable":true,"label":"Cazuri confirmate","format":{"places":0,"digitSeparator":true}},{"fieldName":"SIRUTA_judet","visible":false,"isEditable":true,"label":"SIRUTA_judet","format":{"places":0,"digitSeparator":true}},{"fieldName":"Regiune_dezvoltare","visible":false,"isEditable":true,"label":"Regiune_dezvoltare"},{"fieldName":"Populatie","visible":false,"isEditable":true,"label":"Populatie","format":{"places":2,"digitSeparator":true}},{"fieldName":"Persoane_in_carantina","visible":true,"isEditable":true,"label":"Persoane în carantină","format":{"places":0,"digitSeparator":true}},{"fieldName":"Persoane_izolate","visible":true,"isEditable":true,"label":"Persoane izolate","format":{"places":0,"digitSeparator":true}},{"fieldName":"Persoane_decedate","visible":false,"isEditable":true,"label":"Persoane decedate","format":{"places":0,"digitSeparator":true}},{"fieldName":"Persoane_vindecate","visible":false,"isEditable":true,"label":"Persoane vindecate","format":{"places":0,"digitSeparator":true}},{"fieldName":"Probe_in_asteptare","visible":false,"isEditable":true,"label":"Probe în așteptare","format":{"places":0,"digitSeparator":true}},{"fieldName":"Cazuri_infirmate","visible":false,"isEditable":true,"label":"Cazuri infirmate","format":{"places":0,"digitSeparator":true}},{"fieldName":"GlobalID","label":"GlobalID","isEditable":false,"visible":false},{"fieldName":"EditDate","label":"EditDate","isEditable":false,"visible":false,"format":{"dateFormat":"shortDateShortTime","timezone":"utc"}}],"description":"Persoane izolate: <font color='#4169e1'>{Persoane_izolate}<\/font><span style='font-weight: bold;'><br /><\/span>Persoane în carantină<span style='color: rgb(255, 0, 0);'><span style='font-weight: bold;'><span style='color: rgb(76, 76, 76);'><\/span><\/span><\/span>: <font color='#ffa500'>{Persoane_in_carantina}<\/font><span style='color: rgb(255, 0, 0);'><span style='font-weight: bold;'><br /><\/span><\/span>","showAttachments":false,"mediaInfos":[]}}],"baseMap":{"baseMapLayers":[{"id":"VectorTile_2846","type":"VectorTileLayer","layerType":"VectorTileLayer","title":"Dark Gray Base","styleUrl":"https://cdn.arcgis.com/sharing/rest/content/items/5e9b3685f4c24d8781073dd928ebda50/resources/styles/root.json","visibility":true,"opacity":1},{"id":"VectorTile_1645","type":"VectorTileLayer","layerType":"VectorTileLayer","title":"Dark Gray Reference","styleUrl":"https://cdn.arcgis.com/sharing/rest/content/items/747cb7a5329c478cbe6981076cc879c5/resources/styles/root.json","isReference":true,"visibility":true,"opacity":0}],"title":"Dark Gray Canvas"},"spatialReference":{"wkid":102100,"latestWkid":3857},"authoringApp":"WebMapViewer","authoringAppVersion":"7.4","version":"2.16","tables":[{"url":"https://services7.arcgis.com/I8e17MZtXFDX9vvT/arcgis/rest/services/TimelineCovid_public/FeatureServer/0","id":"TimelineCovid_public_4730","title":"Timeline","itemId":"664077c34bcd4008a7b9f7475d070ea0","popupInfo":{"title":"CazuriTimeline: {Data}","fieldInfos":[{"fieldName":"OBJECTID","label":"OBJECTID","isEditable":false,"tooltip":"","visible":false,"stringFieldOption":"textbox"},{"fieldName":"Data","label":"Data confirmare","isEditable":true,"tooltip":"","visible":true,"stringFieldOption":"textbox","format":{"dateFormat":"shortDateShortTime"}},{"fieldName":"NumarCazuri","label":"Numar cazuri confirmate","isEditable":true,"tooltip":"","visible":true,"stringFieldOption":"textbox","format":{"places":0,"digitSeparator":true}}],"description":null,"showAttachments":true,"mediaInfos":[]}}],"applicationProperties":{"viewing":{"routing":{"enabled":true},"basemapGallery":{"enabled":true},"measure":{"enabled":true}}}}


NameError: name 'true' is not defined